In [ ]:
#Importar URDF
from pathlib import Path
from roboticstoolbox import ERobot, Robot
from pathlib import Path
print("Pasta atual:", Path().absolute())
urdf_path = Path().absolute() / "mont_simplificada.urdf"
#cookbot = Robot.URDF(str(urdf_path))
cookbot, *_ = ERobot.URDF_read(str(urdf_path))


print("Nome do robô:", cookbot.name)
print("Número de juntas:", cookbot.n)
print("q neutro:", cookbot.qn)
print("FK exemplo:", cookbot.fkine(cookbot.qn))

Pasta atual: c:\Users\User\Desktop\Cookbot2.0
Tipo de cookbot: <class 'list'>


In [59]:
#TESTES
from spatialmath import SE3

posicaoinicial = SE3(0, 0, 0.1)
print("Juntas: ", cookbot.q)
FK = cookbot.fkine(posicaoinicial)
print(FK)

AttributeError: 'list' object has no attribute 'q'

In [11]:
#PARÂMETROS DO ROBÔ
cookbot.dynamics()
print(cookbot)

┌──────────────┬───────┬───────────────────────────────┬─────────────────────────────────────────────────────────────┬────┬───────┬────────┬────┐
│      j       │   m   │               r               │                              I                              │ Jm │   B   │   Tc   │ G  │
├──────────────┼───────┼───────────────────────────────┼─────────────────────────────────────────────────────────────┼────┼───────┼────────┼────┤
│        world │  0    │  0,  0,  0                    │  0,  0,  0,  0,  0,  0                                      │  0 │  0    │  0,  0 │  0 │
│ BASE_ROBO_V2 │  1.85 │ -0.0447,  0.138,  2.59e-15    │  0.00907,  0.0121,  0.0121, -1.01e-12,  3.38e-15,  9.06e-16 │  0 │  0    │  0,  0 │  0 │
│       BASE_2 │  2.67 │  0.173,  1.65e-09, -1.95e-08  │  0.00489,  0.0228,  0.0228, -1.08e-10, -5.69e-08,  6.96e-09 │  0 │  0    │  0,  0 │  0 │
│         ELO2 │  6.44 │  0.03, -0.0875, -0.234        │  0.275,  0.238,  0.0462,  9.42e-06, -0.0834,  2.52e-05      │  0 │ 

In [ ]:
#CINEMÁTICA DIRETA SIMBÓLICA

#importar bibliotecas necessárias
from roboticstoolbox import ET
import numpy as np
from spatialmath import SE3
import sympy as sp

#definindo o braço do robô

#parâmetros do braço
a2 = 600 
a3 = 200

e = ET.tz(qlim=[0, 170]) * ET.tz(300) * ET.Rz(qlim=[-np.pi, np.pi]) * ET.tx(a2) * ET.tx(a3) * ET.Rx(qlim=[-np.pi, np.pi]) * ET.tz(100) * ET.tx(100) * ET.tz(100) #aqui ainda é necessário arrumar o end effector, que será uma cesta
ecookbot = ERobot(e)
#variáveis simbólicas
q1, q2, q3 = sp.symbols('q1 q2 q3')
#definindo a cinemática direta simbólica
TE = e.fkine([q1, q2, q3])
print("A cinemática direta simbólica é:")
print(TE)

#CINEMÁTICA INVERSA

#posição inicial das juntas:

#posição desejado do end effector:




sol = ecookbot.ikine_LM(Tdesejado)
print("A solução da cinemática inversa é:")
print(sol)

A cinemática direta simbólica é:
  1.0*cos(q2)  -1.0*sin(q2)*cos(q3) 1.0*sin(q2)*sin(q3) 100.0*sin(q2)*sin(q3) + 800.0*cos(q2)  
  1.0*sin(q2)  1.0*cos(q2)*cos(q3) -1.0*sin(q3)*cos(q2) 800.0*sin(q2) - 100.0*sin(q3)*cos(q2)  
  0            1.0*sin(q3)  1.0*cos(q3)  1.0*q1 + 100.0*cos(q3) + 300.0  
  0            0            0            1.00000000000000  

A solução da cinemática inversa é:
IKSolution: q=[1.15, 0, 0], success=True, iterations=229, searches=8, residual=1.03e-08


In [ ]:
#TRAJETÓRIAS

#definindo a trajetória
pos1 = SE3(800, 0, 400) * SE3.Rx(0)
pos2 = SE3(0, 800, 420) * SE3.Rx(0)

sol1 = cookbot.ikine_LM(pos1)
print("Sucesso:", sol1.success)
if sol1.success:
    print("q =", sol1.q)
else:
    print("❌ IK não encontrou solução válida para pos1")

sol2 = ecookbot.ikine_LM(pos2)
print("Sucesso:", sol2.success)
if sol2.success:
    print("q =", sol2.q)
else:
    print("❌ IK não encontrou solução válida para pos2")


Sucesso: True
q = [2.66453526e-15 2.36798137e-10 1.89438421e-09]
Sucesso: False
❌ IK não encontrou solução válida para pos2
